In [12]:
###
#
# 
#
###

In [ ]:
## TO DO ##
'''
- bessere korrekte St.Dev-Berechnung (nicht einfach Addiern / Mean / **2 rechnen)

'''


In [4]:
import os
import numpy as np
import pandas as pd
from scipy.stats import norm

In [5]:
seasons = [item for item in reversed(os.listdir('data'))]
pos_files = ['Defenses', 'Kickers', 'Quarterbacks', 'ReceivingRushing']
pos_dict = {'Defenses': 'DSt', 'Kickers': 'K', 'Quarterbacks': 'QB', 'RunningBacks': 'RB', 'TightEnds': 'TE', 'WideReceivers': 'WR'}
#print('seasons:  ', seasons, '\npositions:', positions, '\npos_dict:', pos_dict)

In [6]:
# Team Number of Teams
n_teams = 12

# ScoringSystem JedenVerdammtenSonntagLiga
scor_sys = {
    'PassYds': 1/25, 'PassTD': 6, 'Ints': -4, 'Sck': -0.5, 'Bon_PYds3xx': 1, 'Bon_PYds400+': 2, 'Bon_50yTD': 1,
    'RushYds': 0.1, 'RushTD': 6, 'Bon_RshTD40+': 1, 'Bon_RshYds200+': 2,
    'Rec': 1, 'RecYds': 0.1, 'RecTD': 6, 'Bon_RecTD40+': 1, 'Bon_RecYds200+': 2, 
    'Fumble': -3,
    'PAT': 1, 'PATmiss': -1, 'FG': 3, 'FG50+': 5, 'FGmiss20-': -2,
    'DefSck': 1, 'DefInt': 2, 'DefFumRec': 2, 'DefSaf': 2, 'DefTD': 6, 'DefBlockedKicks': 1,
    'PtsAll0': 10, 'PtsAll1-6': 7, 'PtsAll7-13': 4, 'PtsAll14-20': 1, 'PtsAll28-34': -1, 'PtsAll35+': -4,
    'YdsAll100-': 2, 'YdsAll100-200': 1, 'YdsAll450-499': -2,'YdsAll500+': -4 }

In [7]:
# PPR without Bonus
scor_sys = {
    'PassYds': 1/25, 'PassTD': 4, 'Ints': -2, 'Sck': 0, 'Bon_PYds3xx': 0, 'Bon_PYds400+': 0, 'Bon_50yTD': 0,
    'RushYds': 0.1, 'RushTD': 6, 'Bon_RshTD40+': 0, 'Bon_RshYds200+': 0,
    'Rec': 1, 'RecYds': 0.1, 'RecTD': 6, 'Bon_RecTD40+': 0, 'Bon_RecYds200+': 0, 
    'Fumble': -2,
    'PAT': 1, 'PATmiss': -1, 'FG': 3, 'FG50+': 5, 'FGmiss20-': 0,
    'DefSck': 1, 'DefInt': 2, 'DefFumRec': 2, 'DefSaf': 2, 'DefTD': 6, 'DefBlockedKicks': 0,
    'PtsAll0': 10, 'PtsAll1-6': 7, 'PtsAll7-13': 4, 'PtsAll14-20': 1, 'PtsAll28-34': -1, 'PtsAll35+': -4,
    'YdsAll100-': 2, 'YdsAll100-200': 1, 'YdsAll450-499': -2,'YdsAll500+': -4 }

In [8]:
season = '2020'
position = pos_files[3]
path_order_ = os.listdir('./data/2020/'+position)

In [9]:
df = pd.read_csv('./data/2020/'+position+'/'+path_order_[2]) 
cols = list(df.columns)
pos_dict_cols={'Defenses': [0, 2, 3, 4, 5, 6, 7, 9, 11, 13, 14, 15, 16, 18, 19, 20, 24, 25, 30, 31, 32], 
               'Kickers': [0, 1, 3, 4, 6, 9, 12, 15, 18, 19], 
               'Quarterbacks': [0, 1, 8, 11, 13, 14, 15, 22, 23, 25],  # ADoT 10, Adj.Comp% 7
               'RunningBacks': [0, 1, 3, 5, 7, 9, 10, 30, 32, 33, 38],  # Carries 28, Y/Carr 33, YACont/Carr 34, TackleAvoid/Carr 36
               'TightEnds': [0, 1, 3, 5, 7, 9, 10, 30, 32, 33, 38],     # Targets 4, ADoT 13, RAC/Rec 16
               'WideReceivers': [0, 1, 3, 5, 7, 9, 10, 30, 32, 33, 38]} # Targets 4, ADoT 13, RAC/Rec 16

In [10]:
cols_to_keep = [cols[i] for i in [0, 1, 3, 5, 7, 9, 10, 30, 32, 33, 38]]
df.drop(columns=[col for col in df if col not in cols_to_keep], inplace=True)
df.head(5)

,player,team,position,recRec,recYds,recTds,recTd40s,rushYds,rushTds,rushTd40s,fumbles
0,Diontae Spencer,DEN,WR,0,0,0,0,0,0,0,0
1,Jalen Reagor,PHI,WR,4,52,0,0,0,0,0,0
2,Justin Jefferson,MIN,WR,3,86,1,0,0,0,0,0
3,Jerry Jeudy,DEN,WR,3,37,0,0,0,0,0,0
4,A.J. Brown,TEN,WR,4,62,1,0,0,0,0,0


In [229]:
# for a given season and position this function
# iterates over all weeks and keeps only Fanatsy-Points-Relevant Columns
# Then, the weekly columns are stacked together to have one dataframe (note: a Player is up to 16 times in this df)
# Additionally, for every week the mean&sd of the top12, top24 and Rank12-24 are calculated
#    for QBs, TEs, Def, K the FPAR-Value is calculated by the Top12 # for a 12-Team-League
#    for RBs, WRs the FPAR-Value is calculated by the Top24 # for a 12-Team-League

def DataFrameCreater(season, position, cols_to_keep, baseline_team=None, n_teams=12):
    
    if position == 'RunningBacks' or position == 'WideReceivers' or position == 'TightEnds':
        path_order = './data/'+str(season)+'/ReceivingRushing'
    else:
        path_order = './data/'+str(season)+'/'+position
    n_teams_ = int(n_teams * 1.5) # to considere bench/wrong played somewhat # since the top12 are way to high
    
    #initialize data frames: 
    #   df_pos: contains all Players for Pos.
    #   baseline_team: contains means and std of BaselineTeam per Pos and Week (Rows=Weeks)
    df_pos = None
    if baseline_team is None:
        baseline_team = pd.DataFrame(columns=['QB_m', 'QB_sd', 'RB1_m', 'RB1_sd', 'RB2_m', 'RB2_sd','WR1_m', 'WR1_sd', 'WR2_m', 'WR2_sd', 'Flex_m', 'Flex_sd', 'TE_m', 'TE_sd', 'DSt_m', 'DSt_sd', 'K_m', 'K_sd'], 
                                 index=range(1,17))
    flex_m, flex_sd = [], []
    # iterate over weeks
    for file in os.listdir(path_order):
        beg, end = file.find('('), file.find(')') 
        week_nr = int(file[beg+1:end])+1
        
        df = pd.read_csv(path_order+'/'+file)
        cols_ = [df.columns[i] for i in cols_to_keep]
        df.drop(columns=[col for col in df if col not in cols_], inplace=True)
        df.insert(2, 'week', week_nr)
        
        #calc the fantasy points using scor_sys for each position
        if position == 'Quarterbacks':
            #the last term above gives back the bonus points for PassYds: 1 Pt for 300-399yds, 2 for 400+
            df['FP'] = round(df['yds']*scor_sys['PassYds'] + df['tds']*scor_sys['PassTD'] + df['td50s']*scor_sys['Bon_50yTD'] + df['ints']*scor_sys['Ints'] + df['sks']*scor_sys['Sck'] + df['rushYds']*scor_sys['RushYds'] + df['rushTds']*scor_sys['RushTD']+ df['fumbles']*scor_sys['Fumble'] + np.minimum(np.maximum(round(df['yds']/100-2.499,0),0), 2),3)
        if position == 'RunningBacks' or position == 'WideReceivers' or position == 'TightEnds':
            df = df[df['position'] == pos_dict[position]]
            df.drop(columns='position', inplace=True)
            df['FP'] = round(df['recRec']*scor_sys['Rec'] + (df['recYds']+df['rushYds'])*scor_sys['RushYds'] + (df['recTds']+df['rushTds'])*scor_sys['RushTD'] + df['fumbles']*scor_sys['Fumble'] + df['recTd40s']*scor_sys['Bon_RecTD40+'] + df['rushTd40s']*scor_sys['Bon_RshTD40+'] + scor_sys['Bon_RshYds200+']*(np.maximum(round(df['recYds']/100-1.499,0),0)+np.maximum(round(df['rushYds']/100-1.499,0),0)), 3)     
        if position == 'Defenses':
            df['FP'] = round(df['sacks']*scor_sys['DefSck'] + df['interceptions']*scor_sys['DefInt'] + df['fumbleRecoveries']*scor_sys['DefFumRec'] + df['safeties']*scor_sys['DefSaf'] + 6*(df['defenseTouchdowns']+df['puntTouchdowns']+df['kickTouchdowns']) + df['blockedKicks']*scor_sys['DefBlockedKicks'] + df['pa0Games']*scor_sys['PtsAll0'] + df['pa16Games']*scor_sys['PtsAll1-6'] + df['pa713Games']*scor_sys['PtsAll7-13'] + df['pa1420Games']*scor_sys['PtsAll14-20'] + df['pa2834Games']*scor_sys['PtsAll28-34'] + (df['pa3545Games']+df['pa46plusGames'])*scor_sys['PtsAll35+'] + df['ya099Games']*scor_sys['YdsAll100-'] + df['ya100199Games']*scor_sys['YdsAll100-200'] + df['ya450499Games']*scor_sys['YdsAll450-499'] + (df['ya500549Games']+df['ya550plusGames'])*scor_sys['YdsAll500+'], 3)                           
        if position == 'Kickers':
            comp = df['complete19'] + df['complete29'] + df['complete39'] + df['complete49']
            miss19, misseps = df['attempt19']-df['complete19'], df['epsAttempt']-df['epsMade']
            df['FP'] = round(comp*scor_sys['FG'] + df['complete50']*scor_sys['FG50+'] + df['epsMade']*scor_sys['PAT'] + misseps*scor_sys['PATmiss'] + miss19*scor_sys['FGmiss20-'],3)
          
        # define the mean, std of the top-ranked players in the resp. week   
        top18_m, top18_sd = round(np.sort(df['FP'])[::-1][0:n_teams_].mean(), 3), round(np.sort(df['FP'])[::-1][0:n_teams].std(), 3)
        top36_m           = round(np.sort(df['FP'])[::-1][0:n_teams_*2].mean(), 3)
        t18_36_m, t18_36_sd = round(np.sort(df['FP'])[::-1][n_teams_:n_teams_*2].mean(), 3), round(np.sort(df['FP'])[::-1][n_teams_:n_teams_*2].std(), 3)
    
        # calc FantPtsAboveReplacement
        # write values to baseline team
        if (position == 'RunningBacks' or position == 'WideReceivers'):
            wr_rb_repl = np.sort(df['FP'])[::-1][n_teams*2]
            df['FPAX'] = round(df['FP'] - wr_rb_repl, 3)
            #df['FPAX'] = round(df['FP'] - top36_m, 3)
            p1 = pos_dict[position]+'1_'
            p2 = pos_dict[position]+'2_'
            baseline_team[p1+'m'].loc[week_nr] = top18_m
            baseline_team[p2+'m'].loc[week_nr] = t18_36_m
            baseline_team[p1+'sd'].loc[week_nr] = top18_sd
            baseline_team[p2+'sd'].loc[week_nr] = t18_36_sd
            flex_m.append(round(np.sort(df['FP'])[::-1][n_teams_*2:n_teams_*3].mean(), 3))
            flex_sd.append(round(np.sort(df['FP'])[::-1][n_teams_*2:n_teams_*3].std(), 3))
        else: 
            qb_te_repl = np.sort(df['FP'])[::-1][n_teams]
            df['FPAX'] = round(df['FP'] - qb_te_repl, 3)            
            #df['FPAX'] = round(df['FP'] - top18_m, 3)            
            p = pos_dict[position]+'_'
            baseline_team[p+'m'].loc[week_nr] = top36_m
            baseline_team[p+'sd'].loc[week_nr] = top18_sd
        
        # append dataframe
        if df_pos is None:
            df_pos = df
        else:
            df_pos = df_pos.append(df, ignore_index=True)

    return df_pos, baseline_team, (flex_m, flex_sd)

In [230]:
# Calculate FWPctAX for each Player per Week and store in CSV (inplace)

def CalcFWAR(season, position, baseline_team):
    
    norm_table = []
    for week in range(16):
        norm_table.append(norm(float(baseline_team['Team_m'].iloc[week]), float(baseline_team['Team_sd'].iloc[week])))
    
    df = pd.read_csv('data/'+str(season)+'/'+position+'.csv', sep=';', decimal=',', index_col=0)
    df['FWPctAX'] = np.nan
    
    for idx in range(df.shape[0]):
        week = df['week'].iloc[idx] - 16
        val = float(baseline_team['Team_m'].iloc[week]) + float(df['FPAX'].iloc[idx])
        df['FWPctAX'].iloc[idx] = norm_table[week].cdf(val)
        
    df.to_csv('data/'+str(season)+'/'+position+'_ppr.csv', sep=';', decimal=',')
    
    return df

In [231]:
#season = '2020'
n_teams = 12

for season in seasons:
    baseline_team = None
    
    for pos in list(pos_dict.keys()):
        cols_to_keep = pos_dict_cols[pos]
        if pos == 'RunningBacks':
            df, baseline_team, flex_rb = DataFrameCreater(season, pos, cols_to_keep, baseline_team)
        elif pos == 'WideReceivers': 
            df, baseline_team, flex_wr = DataFrameCreater(season, pos, cols_to_keep, baseline_team)
        else: 
            df, baseline_team, _ = DataFrameCreater(season, pos, cols_to_keep, baseline_team)
        
        df.to_csv('data/'+str(season)+'/'+pos+'_ppr.csv', sep=';', decimal=',')

    # Flex
    baseline_team['Flex_m'] = (np.array(flex_rb[0])+np.array(flex_wr[0]))/2
    baseline_team['Flex_sd'] = np.array(flex_rb[1])+np.array(flex_wr[1])

    # Team
    team_vals_m = baseline_team['QB_m']+baseline_team['RB1_m']+baseline_team['RB2_m']+baseline_team['WR1_m']+baseline_team['WR2_m']+baseline_team['TE_m']+baseline_team['Flex_m']+baseline_team['DSt_m']+baseline_team['K_m']
    team_vals_sd = ((baseline_team['QB_sd']+baseline_team['RB1_sd']+baseline_team['WR1_sd']+baseline_team['TE_sd']+baseline_team['DSt_sd']+baseline_team['K_sd'])/6) ** 2
    baseline_team.insert(0, 'Team_m', team_vals_m)
    baseline_team.insert(1, 'Team_sd', team_vals_sd)

    baseline_team.to_csv('data/'+str(season)+'/baseline_team_ppr.csv', sep=';', decimal=',')


In [232]:
# iterate over seasons and position to add the weekly FantasyWinsAboveReplacement
for season in seasons:
    baseline_team = pd.read_csv('data/'+str(season)+'/baseline_team_ppr.csv', sep=';', decimal=',', index_col=0)
    for pos in list(pos_dict.keys()):
        _ = CalcFWAR(season, pos, baseline_team=baseline_team)

C:\Users\lordm\.conda\envs\LMT\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [233]:
# Create a Dataframe for all Players of a Season
# with their weekly FantasyPts FP, FPAR, FWPctAR and moreover the seasonal value (MW bzw MW*16)
# drop all other stats-columns

# überlege, ob MW oder Summe/15 (16G - Bye) nehme -> Ausfälle in Betracht genommen z.T.
# weil nur Spiele, die er gespielt hat, hat er auch Wins gebracht eig

def Players_df(season):
    df_all = None
    
    for pos in reversed(list(pos_dict.keys())): # reversed, so we dont start with Def
    
        df = pd.read_csv('data/'+str(season)+'/'+pos+'_ppr.csv', sep=';', decimal=',', index_col=0)
        df.insert(2, 'position', pos_dict[pos])
        df['FWPctAX'] = np.round(df['FWPctAX'],3)
        cols_to_keep = ['player', 'team', 'position', 'week', 'FP', 'FPAX', 'FWPctAX']
        df.drop(columns=[col for col in df if col not in cols_to_keep], inplace=True)
        
        if pos == 'Defenses':
            df['player'] = df['team']
        
        if df_all is None:
            df_all = df
        else:
            df_all = df_all.append(df, ignore_index=True)
    
    return df_all

for season in seasons:
    df = Players_df(season)
    df.to_csv('data/'+str(season)+'/All_Players_Fts_ppr.csv', sep=';', decimal=',')

In [234]:
# df_seasonal: every row contains a player with his team, pos
#              FP_total = summed Fantasy-Points for player (Ftp total of Season -> less if player missed time)
#              FP_mean = mean Fantasy-Points for player (per Game PLAYED)
#              FPAR_total = summed FPtsAboveReplacement for player (Ftp total of Season -> less if player missed time)
#              FPAR_mean = mean FPtsAboveReplacement for player (per Game PLAYED)
#              FWins_actualadded = summed FWinsAboveReplacement for player (FWins of Season -> less if player missed time)
#              FWAR_mean = mean FWinsAboveReplacement for player (FWins per Game PLAYED)
#

for season in seasons:
    # load DF for all players
    df = pd.read_csv('data/'+str(season)+'/All_Players_Fts_ppr.csv', sep=';', decimal=',', index_col=0)
    player_list = np.unique(list(df['player']))
    # initialize dataFrame
    df_seasonal = pd.DataFrame(columns=['player', 'team', 'position', 'FP_total','FP_meanGame', 'FPAX_total','FPAX_meanGame', 'FWins_IfFull_added', 'FWins_actual_added', 'FWAR'], index = range(len(player_list)))
    #df_seasonal = pd.DataFrame(columns=['player', 'team', 'position', 'FP_total','FP_meanGame', 'FPAX_total', 'FWins_added', 'FWAR'], index = range(len(player_list)))

    # Calc Replacement Wins for Position
    # -> Average Player from Baseline-Team (for RB take Avg of RB1&RB2; same for WR)
    # Dann: Replacement-value = 0.5*15
    # Nein, siehe zB TE: 4. bester hat neg FWins_actual_added bereits
    #(Alternative: take 12./24. best - calculated below)

    # iterate over players:
    for i, player in enumerate(player_list):  
        df_player = df[df['player'] == player]
        df_seasonal.iloc[i, 0:3] = df_player[['player', 'team', 'position']].iloc[0]
        df_seasonal['FP_total'].iloc[i] = round(df_player['FP'].sum(), 3)
        df_seasonal['FP_meanGame'].iloc[i] = round(df_player['FP'].mean(), 3)
        # for missed games add expPoints 0 and FWinPct 0.5
        # FALSCH: hier sollte xPts sowas wie Baseline - Repl sein
        #np_fpax  = np.append(np.array(df_player['FPAX']), np.zeros(max(15-df_player.shape[0],0))) 
        #np_fwpct = np.append(np.array(df_player['FWPctAX']), np.ones(max(15-df_player.shape[0],0))*0.5)
        df_seasonal['FPAX_total'].iloc[i] = round(df_player['FPAX'].sum(), 3)
        df_seasonal['FPAX_meanGame'].iloc[i]  = round(df_player['FPAX'].mean(), 3)
        df_seasonal['FWins_IfFull_added'].iloc[i] = round(df_player['FWPctAX'].mean()*15,3)  # over Season -> times 15 for 16 Games minus Bye Week 
        df_seasonal['FWins_actual_added'].iloc[i] = round(df_player['FWPctAX'].sum(), 3) # influence only in the Games he played
        #df_seasonal['FWins_added'].iloc[i] = round(np_fwpct.sum(), 3)

    pos_fwar = {}
    for pos in list(pos_dict.values()):
        if pos == 'RB' or pos == 'WR':
            repl_val = df_seasonal[df_seasonal['position']== pos].sort_values('FWins_actual_added', axis=0, ascending=False)['FWins_actual_added'].iloc[24] 
            #repl_val = df_seasonal[df_seasonal['position']== pos].sort_values('FWins_added', axis=0, ascending=False)['FWins_added'].iloc[24] 
            pos_fwar[pos] = repl_val
        else:
            repl_val = df_seasonal[df_seasonal['position']== pos].sort_values('FWins_actual_added', axis=0, ascending=False)['FWins_actual_added'].iloc[12]
            #repl_val = df_seasonal[df_seasonal['position']== pos].sort_values('FWins_added', axis=0, ascending=False)['FWins_added'].iloc[12]
            pos_fwar[pos] = repl_val

    for i, player in enumerate(player_list): 
        pos = df_seasonal['position'].iloc[i]
        df_seasonal['FWAR'].iloc[i] = round(df_seasonal['FWins_actual_added'].iloc[i] - pos_fwar[pos],3)
        #df_seasonal['FWAR'].iloc[i] = round(df_seasonal['FWins_added'].iloc[i] - pos_fwar[pos],3)
    
    for col in df_seasonal.columns:
        if not col in ['player', 'team', 'position']:
            df_seasonal[col] = [float(d) for d in df_seasonal[col]]
    
    df_seasonal.to_csv('data/'+str(season)+'/FantasyWinsAboveReplacement_PPR.csv', sep=';', decimal=',')
    break

In [11]:
df_seasonal = pd.read_csv('data/'+str(2020)+'/FantasyWinsAboveReplacement_PPR.csv', sep=';', decimal=',', index_col=0)

In [12]:
df_2020 = pd.read_csv('data/'+str(2020)+'/All_Players_Fts_ppr.csv', sep=';', decimal=',', index_col=0)
df_2020[df_2020['player']=='A.J. Brown']

,player,team,position,week,FP,FPAX,FWPctAX
3,A.J. Brown,TEN,WR,1,8.9,-10.308,0.316
313,A.J. Brown,TEN,WR,11,16.2,-2.633,0.465
547,A.J. Brown,TEN,WR,12,19.8,0.681,0.512
605,A.J. Brown,TEN,WR,13,8.7,-10.814,0.248
747,A.J. Brown,TEN,WR,14,24.2,6.225,0.681
912,A.J. Brown,TEN,WR,15,15.4,-2.469,0.460
1063,A.J. Brown,TEN,WR,16,8.3,-11.564,0.234
1519,A.J. Brown,TEN,WR,5,21.2,1.689,0.522
1656,A.J. Brown,TEN,WR,6,22.8,5.689,0.576
1795,A.J. Brown,TEN,WR,7,27.3,6.586,0.592


In [13]:
fwar = df_seasonal.sort_values('FWAR', axis=0, ascending=False)
fwar.head()

,player,team,position,FP_total,FP_meanGame,FPAX_total,FPAX_meanGame,FWins_IfFull_added,FWins_actual_added,FWAR
27,Alvin Kamara,NO,RB,375.8,25.053,142.496,9.500,9.816,9.816,4.205
663,Travis Kelce,KC,TE,310.6,20.707,99.260,6.617,9.436,9.436,3.395
165,Dalvin Cook,MIN,RB,327.8,23.414,108.868,7.776,9.425,8.797,3.186
189,Davante Adams,GB,WR,341.8,26.292,97.280,7.483,9.710,8.415,2.743
622,Stefon Diggs,BUF,WR,314.0,20.933,29.766,1.984,8.256,8.256,2.584


In [14]:
draft = pd.read_csv('Draft\DraftHistory2020.csv', sep=';')
fwar_stripped = fwar.drop(columns=['team', 'position', 'FP_meanGame', 'FPAX_total', 'FPAX_meanGame','FWins_IfFull_added','FWins_actual_added'])
draft_fwar = draft.merge(fwar_stripped, how='left', on='player')

In [16]:
draft_fwar.to_csv('draft_fwar.csv', sep=';', decimal=',')
draft_fwar

,Rank,player,Pos,Team,FantTeam,FantPlayer,FP_total,FWAR
0,1,Christian McCaffrey,RB,CAR,teamtilli,fabio,90.40,-3.305
1,2,Derrick Henry,RB,TEN,Blue Mountain Goats,Matthias,294.10,2.550
2,3,Saquon Barkley,RB,NYG,America's Team,Nico,15.40,-4.897
3,4,Ezekiel Elliott,RB,DAL,Showdown United,Michael,204.60,1.049
4,5,Dalvin Cook,RB,MIN,Cowboys,Andreas,327.80,3.186
...,...,...,...,...,...,...,...,...
175,176,Jared Goff,QB,LA,Shotguns,Lorenz,226.48,-0.325
176,177,LA,DSt,LA,Riparian Meadow Ravens,Felix,128.00,1.005
177,178,Jimmy Garoppolo,QB,SF,Weisskirchner Slugs,Michael,54.84,-4.019
178,179,Tyler Eifert,TE,JAX,Geidorf Drunkards,Stefan,80.90,-1.241


In [21]:
draft_fwar.drop(columns = ['Rank', 'Team', 'FantPlayer']).groupby('FantTeam').sum().sort_values('FWAR')

,FP_total,FWAR
FantTeam,,
Blue Mountain Goats,1503.12,-23.359
America's Team,1839.92,-20.553
Drunken Mermaids,1983.48,-19.621
Riparian Meadow Ravens,1864.80,-18.085
Suicide squad,2199.76,-10.637
Cowboys,2226.58,-9.724
teamtilli,2424.18,-7.322
Geidorf Drunkards,2481.06,-4.302
Weisskirchner Slugs,2393.68,-3.169
